# Data Preprocessing

In this notebook we will use the original file `Historical_Product_Demand.csv` to create a Date-Product pivot table `Historical_Product_Demand_per_Product.csv` that can be used for modeling.

The steps we follow to do that are: read in the data, clean up the format of order demand and convert it from strings to numbers, build a pivot table for each warehouse with the demand for each product-date combination, filling all null combinations with 0s. Then we combine all the pivot tables into one and filter out a subset of the data for final data clean up. Once we have the dataset of our choice, we save it back to the project.

## Step 1: Read in the data

Read in the historical data from `/project_data/data_asset/Historical_Product_Demand.csv`

In [1]:
import numpy as np
import pandas as pd

df_data_1 = pd.read_csv('/project_data/data_asset/Historical_Product_Demand.csv')
df_data_1.head()

Product_Code Warehouse Product_Category       Date Order_Demand
0  Product_0993    Whse_J     Category_028  2012/7/27         100 
1  Product_0979    Whse_J     Category_028  2012/1/19         500 
2  Product_0979    Whse_J     Category_028   2012/2/3         500 
3  Product_0979    Whse_J     Category_028   2012/2/9         500 
4  Product_0979    Whse_J     Category_028   2012/3/2         500

Look at some aggregated information about the dataset.

In [2]:
df_data_1.describe()

Product_Code Warehouse Product_Category       Date Order_Demand
count        1048575   1048575          1048575    1037336      1048575
unique          2160         4               33       1729         3828
top     Product_1359    Whse_J     Category_019  2013/9/27        1000 
freq           16936    764447           481099       2075       112682

## Step 2: Prepare Data

### Clean the Order Demand values

Some order demand values are not formatted correctly. Values such as `(100)` might represent returns or some kind of backwards flow of items, so we represent those values with a negative sign such as `-100`. Once we resolve what the string should look like, we convert it into an integer type.

In [3]:
df_data_1['Order_Demand'] = df_data_1['Order_Demand'].str.strip(' ')
df_data_1['Order_Demand'] = df_data_1['Order_Demand'].str.replace('(', '-')
df_data_1['Order_Demand'] = df_data_1['Order_Demand'].str.replace(')', '')

df_data_1['Order_Demand'] = df_data_1['Order_Demand'].astype(int)

### Build Pivot Tables for Warehouses

In this loop, we iterate through all the unique warehouse names and build a pivot table for each warehouse with the demand for each product-date combination, filling all null combinations with 0s. We also concatenate them all into one table as we go.

In [4]:
# Replace missing values with 0

wh_map = {'Whse_A': 'W1', 'Whse_J': 'W2', 'Whse_C': 'W3', 'Whse_S': 'W4'}

for wh in df_data_1['Warehouse'].unique():
    if wh == df_data_1['Warehouse'].unique()[0]:
        pivot_table_main = pd.pivot_table(df_data_1[df_data_1['Warehouse'] == wh], values='Order_Demand', index='Date', columns='Product_Code', aggfunc=np.sum, fill_value=0)[:]
        pivot_table_main.columns = pivot_table_main.columns + '_' + wh_map[wh]
        pivot_table_main.index = pd.to_datetime(pivot_table_main.index)
    else:
        pivot_table = pd.pivot_table(df_data_1[df_data_1['Warehouse'] == wh], values='Order_Demand', index='Date', columns='Product_Code', aggfunc=np.sum, fill_value=0)[:]
        pivot_table.columns = pivot_table.columns + '_' + wh_map[wh]
        pivot_table.index = pd.to_datetime(pivot_table.index)
        pivot_table_main = pd.concat([pivot_table_main, pivot_table], axis=1, join='outer')

### Apply Date Filters

To perform some final clean up we filter out certain dates from the data.

In [5]:
final_df = pivot_table_main[42:].sort_index()

In [6]:
# Limit the data to certain time range if needed
date_list = pd.date_range(start = '2012-01-01', end = '2017-01-09').difference(final_df.index)

In [7]:
final_df = pd.concat([final_df, pd.DataFrame(0, index=date_list, columns=final_df.columns)], axis=0).sort_index()

In [8]:
# Replace missing values with 0

final_df = final_df.fillna(0)

## Step 3: Preview Final Data

We can preview the dataset before saving it back to the project.

In [9]:
final_df.head()

Product_Code  Product_0001_W2  Product_0003_W2  Product_0004_W2  \
2012-01-01                0.0              0.0              0.0   
2012-01-02                0.0              0.0              0.0   
2012-01-03                0.0              0.0              0.0   
2012-01-04                0.0              0.0              0.0   
2012-01-05              100.0              0.0              0.0   

Product_Code  Product_0005_W2  Product_0006_W2  Product_0007_W2  \
2012-01-01                0.0              0.0              0.0   
2012-01-02                0.0              0.0              0.0   
2012-01-03                0.0              0.0              0.0   
2012-01-04                0.0              0.0              0.0   
2012-01-05                0.0              0.0              0.0   

Product_Code  Product_0008_W2  Product_0009_W2  Product_0010_W2  \
2012-01-01                0.0              0.0              0.0   
2012-01-02                0.0              0.0              0.0   
2012-01-03                0.0              0.0              0.0   
2012-01-04                0.0              0.0              0.0   
2012-01-05             3000.0              0.0              0.0   

Product_Code  Product_0011_W2  ...  Product_2138_W1  Product_2142_W1  \
2012-01-01                0.0  ...              0.0              0.0   
2012-01-02                0.0  ...              0.0              0.0   
2012-01-03                0.0  ...            144.0              0.0   
2012-01-04                0.0  ...            350.0              0.0   
2012-01-05             1400.0  ...              0.0              0.0   

Product_Code  Product_2144_W1  Product_2147_W1  Product_2164_W1  \
2012-01-01                0.0              0.0              0.0   
2012-01-02                0.0              0.0              0.0   
2012-01-03                0.0              0.0              0.0   
2012-01-04                0.0              0.0              0.0   
2012-01-05                0.0              0.0              0.0   

Product_Code  Product_2165_W1  Product_2166_W1  Product_2167_W1  \
2012-01-01                0.0              0.0              0.0   
2012-01-02                0.0              0.0              0.0   
2012-01-03                0.0              0.0              0.0   
2012-01-04               10.0              0.0              0.0   
2012-01-05                0.0              0.0             17.0   

Product_Code  Product_2168_W1  Product_2169_W1  
2012-01-01                0.0              0.0  
2012-01-02                0.0              0.0  
2012-01-03                0.0              0.0  
2012-01-04                0.0              0.0  
2012-01-05                0.0              0.0  

[5 rows x 2849 columns]

In [10]:
final_df.tail()

Product_Code  Product_0001_W2  Product_0003_W2  Product_0004_W2  \
2017-01-05                0.0              0.0              0.0   
2017-01-06                0.0              0.0              0.0   
2017-01-07                0.0              0.0              0.0   
2017-01-08                0.0              0.0              0.0   
2017-01-09                0.0              0.0              0.0   

Product_Code  Product_0005_W2  Product_0006_W2  Product_0007_W2  \
2017-01-05                0.0              0.0              0.0   
2017-01-06                0.0              0.0              0.0   
2017-01-07                0.0              0.0              0.0   
2017-01-08                0.0              0.0              0.0   
2017-01-09                0.0              0.0              0.0   

Product_Code  Product_0008_W2  Product_0009_W2  Product_0010_W2  \
2017-01-05                0.0              0.0              0.0   
2017-01-06                0.0              0.0              0.0   
2017-01-07                0.0              0.0              0.0   
2017-01-08                0.0              0.0              0.0   
2017-01-09                0.0              0.0              0.0   

Product_Code  Product_0011_W2  ...  Product_2138_W1  Product_2142_W1  \
2017-01-05                0.0  ...              0.0              0.0   
2017-01-06                0.0  ...              0.0              0.0   
2017-01-07                0.0  ...              0.0              0.0   
2017-01-08                0.0  ...              0.0              0.0   
2017-01-09                0.0  ...              0.0              0.0   

Product_Code  Product_2144_W1  Product_2147_W1  Product_2164_W1  \
2017-01-05                0.0              0.0              0.0   
2017-01-06                0.0              0.0              0.0   
2017-01-07                0.0              0.0              0.0   
2017-01-08                0.0              0.0              0.0   
2017-01-09                0.0              0.0              0.0   

Product_Code  Product_2165_W1  Product_2166_W1  Product_2167_W1  \
2017-01-05                0.0              0.0              0.0   
2017-01-06                0.0              0.0              0.0   
2017-01-07                0.0              0.0              0.0   
2017-01-08                0.0              0.0              0.0   
2017-01-09                0.0              0.0              0.0   

Product_Code  Product_2168_W1  Product_2169_W1  
2017-01-05                0.0              0.0  
2017-01-06                0.0              0.0  
2017-01-07                0.0              0.0  
2017-01-08                0.0              0.0  
2017-01-09                0.0              0.0  

[5 rows x 2849 columns]

In [11]:
final_df.shape

(1836, 2849)

### Save data back to the project

In [12]:
final_df.to_csv('/project_data/data_asset/Historical_Product_Demand_per_Product.csv')

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020 <br>
All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**